In [ ]:
from sirius import DFT_ground_state_find
from sirius.edft import FreeEnergy
import sirius.baarman as st
import sirius.ot as ot
from numpy import linspace
import matplotlib.pyplot as plt

In [ ]:
def geodesic(X, Y, tau):
    """
    Keyword Arguments:
    X   -- intial point
    Y   -- tangent vector
    tau -- step size
    """
    U, _ = st.stiefel_transport_operators(Y, X, tau)
    return U@X


def p(X, Y, tau, E):
    """
    Compute energy along geodesic
    Keyword Arguments:
    X   -- initial point
    Y   -- tangent vectors
    tau -- step size
    """
    return E(geodesic(X, Y, tau))

In [ ]:
# run a single SCF iteration to initialize the system
res = DFT_ground_state_find(1, config='sirius.json')
# extract wrappers from C++
density = res['density']
potential = res['potential']
hamiltonian = res['hamiltonian']
kset = res['kpointset']

# k-point weights
kw = kset.w
# Hamiltonian, provides gradient H|Ψ>
H = ot.ApplyHamiltonian(hamiltonian, kset)
# create object to compute the total energy
E = ot.Energy(kset, potential, density,
              ot.ApplyHamiltonian(hamiltonian, kset))
# get PW coefficients from C++
X = kset.C
# get occupation numbers
fn = kset.fn

_, HX = E.compute(X)
# ∇E(X)
dAdC = HX*fn*kw
# project gradient of the free energy to the Stiefel manifold
Y = st.stiefel_project_tangent(-dAdC, X)
# evaluate energy along geodesic
ts = linspace(0, 1.5, 20)
es = [p(X, Y, t, lambda X: E(X)) for t in ts]

plt.plot(ts, es, '-x')
plt.ylabel('Energy [Ha]')
plt.xlabel(r'$\tau$')
plt.title(r'Energy along geodesic $X(\tau)$')
plt.show()